In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [31]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

import pickle
with open('train_moong.pickle', 'rb') as f:
    train_x = pickle.load(f) # 단 한줄씩 읽어옴
with open('train_label_moong.pickle', 'rb') as f:
    train_y = pickle.load(f) # 단 한줄씩 읽어옴
with open('test_moong.pickle', 'rb') as f:
    test_x = pickle.load(f) # 단 한줄씩 읽어옴
with open('test_label_moong.pickle', 'rb') as f:
    test_y = pickle.load(f) # 단 한줄씩 읽어옴


In [32]:
# 긍정 / 혹은 부정 문장리스트
def wordCount(data, word):
    cnt = 0;
    for sentence in data:
        if word in sentence:
            cnt += 1
    return cnt;

# data : 긍/ 부정 문서 리스트 
# data[0] : 부정 문서 싹다.
# data[1] : 긍정 문서 싹다. 
# w : 단어 
# c : 긍정, 부정
# return 카이제곱 통계량
def getKai(datas, word, isPositive):
    A = wordCount(datas[isPositive], word)
    B = wordCount(datas[(isPositive+1)%2], w)
    C = len(datas[isPositive]) - A
    D = len(datas[(isPositive+1)%2]) - B
    # 분모가 0이면 0을 리턴함.
    if ((A+B)*(A+C)*(B+D)*(C+D)) == 0:
        return 0
    return ((A+B+C+D)*((A*D-B*C)*(A*D-B*C))) / ((A+B)*(A+C)*(B+D)*(C+D))
train_x = list(train_x)
train_y = list(train_y)
test_x = list(test_x)
test_y = list(test_y)

In [33]:
# words 는 현이님이 주신 피클의 모든 단어들
words = []
for s in train_x:
    for w in s:
        words.append(w)
# 중복 제거
words = list(set(words))
# 긍/부정 문서로 나눈다.
data, dataP, dataN = [], [], []
for i in range(len(train_x)):
    if(train_y[i] == 1):
        dataP.append(train_x[i])
    else:
        dataN.append(train_x[i])
data =[dataN, dataP]

, '정도'], ['내', '가본', '짐', '캐리', '영화', '중', '가장', '연기', '뛰어난', '영화'], ['볼때', '울', '되는', '작품'], ['긴장감', '스릴', '감', '무지무지', '잼', '났어요'], [], ['송지효', '배드', '신은', '점', '주진모', '외', '사랑', '만점'], ['밑', '어떤', '사람', '쓴대로', '달리', '만나러', '갔다가', '페데리코', '만나고', '돌아왔다'], ['와우', '평점', '높네', '점', '대', 'ㅋㅋ'], ['누가', '월레스', '그로', '밋', '점', '주지', '않을수가', '있는가'], ['암울한', '미래', '대한', '한', '측면', '은근', '히', '긴장', '되고', '스릴', '있게', '봤네요', '역시', '미래', '소년', '코난', '잴', '재밌어요'], ['나', '굉장히', '재미있게', '봤다', '타짜', '더', '강렬하던데', '배우', '들', '쌈빡하게', '연기'], ['아이', '함께', '보기', '좋네요'], ['좋은', '영화', '우리', 'R', 'F', '가', '보고싶다'], ['두주니', '재밋더', '고요', '제', '첨', '본', '드라마', 'ㅎ'], ['전체', '적', '흐름', '어딘가', '빠진것처럼', '부족함', '느껴졌지만', '그것', '고전', '영화', '매력', '다가오는', '영화', '원주민', '흔', '치', '않은', '캐릭터', '속', '잘', '녹아난', '우무', '카이', '용맹함과', '캐서린', '향', '순박한', '사랑', '매력', '적', '이었다'], ['대단하가', '개단', '해', '대단하다', '대단해'], ['예전', '봤던건데', '기억', '잘안나네', '재밋'], ['김창완', '대', '풍', '미한', '김완선', '걸', '출한', '댄스', '가수', '키워', '낸', '작곡가', '김창완', '동

In [34]:
kai_dict = []
for w in tqdm(words):
    kai_dict.append(getKai(data, w, 0))
kaiRate = pd.DataFrame()
kaiRate['words'] = words
kaiRate.index = words
kaiRate['word_kai'] = kai_dict
# kaiRate은 카이제곱 정렬 상위 단어 
kaiRate = kaiRate.sort_values(by=['word_kai'],ascending=False)

100%|██████████| 101188/101188 [1:35:37<00:00, 17.64it/s]


In [37]:
const_kai   = 7000   #구할 카이제곱상위 단어의 개수
realkaiRate = kaiRate[:const_kai]
realkaiRate = list(realkaiRate["words"])

with open('kaiRate_train_moong_ordered.pickle', 'wb') as f:
    pickle.dump(kaiRate, f)

In [107]:
const_kai   = 101188   #구할 카이제곱상위 단어의 개수
realkaiRate = kaiRate[:const_kai]
realkaiRate = list(realkaiRate["words"])

In [108]:
v2 = CountVectorizer()
# v2 = CountVectorizer()
# # 기본단어
# cntVectorizer_words = v1.fit(words)
# 카이제곱 상위 단어
cntVectorizer_kai = v2.fit(realkaiRate)

In [109]:
import pickle
with open('train_moong.pickle', 'rb') as f:
    train_x = pickle.load(f) # 단 한줄씩 읽어옴
with open('train_label_moong.pickle', 'rb') as f:
    train_y = pickle.load(f) # 단 한줄씩 읽어옴
with open('test_moong.pickle', 'rb') as f:
    test_x = pickle.load(f) # 단 한줄씩 읽어옴
with open('test_label_moong.pickle', 'rb') as f:
    test_y = pickle.load(f) # 단 한줄씩 읽어옴

In [110]:
xx = []
for s in tqdm(train_x):
    ss = ""
    for w in s:
        ss += (w + " ")
    xx.append(ss)
train_x = xx

xx = []
for s in tqdm(test_x):
    ss = ""
    for w in s:
        ss += (str(w) + " ")
    xx.append(ss)
test_x = xx

# 카이제곱 상위 단어 feature
k_train = cntVectorizer_kai.transform(train_x)

k_test = cntVectorizer_kai.transform(test_x)

100%|██████████| 49157/49157 [00:00<00:00, 231819.31it/s]


In [111]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

classifier = MultinomialNB()

classifier.fit(k_train, train_y)

pred = classifier.predict(k_test).tolist()

print('Accuracy1: %.10f' % accuracy_score(test_y, pred))

Accuracy1: 0.8410399333
